# XGBOOST REGRESSION

In [12]:
!pip install xgboost

In [1]:
# MacOS kullanıcılar yükleme yaparken bir hata ile karşılaşacaklar. Bu hatayı giderebilmek adına terminalden aşağıdaki kodu çalıştırmaları gerekir. 
# sudo xcodebuild -license
# Sonrasında birkaç defa enter yada space e basıp sonrasında q tuşuna basıp çıkılması gerekiyor. 
# Sonrasında gelen ekranda agree yazarak işlem tamamlanmalıdır.
# Bu işlemi yetki vermek olarak düşünebilirsiniz.

In [2]:
# Windows kullanıcılarında notebook çalıştırılmadan önce Anaconda kullanıyorsanız yönetici olarak çalıştırmanız gerekir. VsCode kullanıyorsanız yine yönetici olarak çalıştırmanız gerekir.

In [1]:
import xgboost

In [2]:
import numpy as np 
import pandas as pd 
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn import model_selection

In [4]:
df = pd.read_csv("verisetleri\Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
xgb = XGBRegressor().fit(X_train, y_train)

In [7]:
xgb

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
y_pred = xgb.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

355.4651481224188

## Model tunning

In [11]:
xgb = XGBRegressor()
xgb

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)

In [12]:
# learning_rate --> Öğrenme oranı : Overfitting i engellemek adına kullanılan bir hiperparametredir. Daraltma adım boyunu ifade etmektedir.
# colsample_bytree --> Oluşturulacak olan ağaçlarda değişkenlerden alınacak olan alt küme oranını ifade ediyor.
xgb_params = {"learning_rate" : [0.1, 0.01, 0.5], "max_depth" : [2, 3, 4, 5, 8], "n_estimators" : [100, 200, 500, 1000], "colsample_bytree" : [0.4, 0.7,1]}

In [13]:
xgb_cv_model = GridSearchCV(xgb, xgb_params, cv=10, n_jobs = -1, verbose = 2).fit(X_train, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:   52.2s
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  1.2min finished


In [14]:
xgb_cv_model.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 2,
 'n_estimators': 1000}

In [16]:
xgb_tuned_model = XGBRegressor(colsample_bytree = 0.4, learning_rate = 0.1, max_depth = 2, n_estimators = 1000).fit(X_train, y_train)

In [17]:
# Elimizdeki veri setlerinin boyutlar ve yapısı elimizdeki algoritmaların yapısı ile bazen örtüşmeyebilir. Yani örneğin çok kuvvetli bir algoritma çok kolay basit bir veriseti üzerinde kullanılmaya çalışıldığında  algoritma kendi komplex problemlere uyarlanmak için oluşturulan yapısı itibariyle o problemlerde çok çok iyi sonuçlar gösteremeyebilir. Tam tersi örneğin KNN gibi görece biraz daha basit problemelerde iyi çalışan bir algoritma çok komplex veri setleri için iyi çalışmayabilir. 

In [18]:
y_pred = xgb_tuned_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

367.8515297956715